The [dataset](https://jmcauley.ucsd.edu/data/amazon_v2/index.html) is metadata for Magazine Subscriptions category

In [1]:
import sys

IN_COLAB = 'google.colab' in sys.modules
DIR_PATH = ""

if IN_COLAB:
    from google.colab import drive
    drive.mount("/content/dirve")
    DIR_PATH = r"/content/dirve/MyDrive/研究所/Data/dblp/"
    sys.path.append('/content/dirve/MyDrive/Colab Notebooks/package')
else:
    DIR_PATH = r"D:\\論文實驗\\data\\dblp\\"
    sys.path.append('D:\\論文實驗\\package')
    sys.path.append("D:\\論文實驗\\env\\Lib\\site-packages")

In [ ]:
import pandas as pd
from utils import preprocessingText

DIR_PATH = r"D:\\論文實驗\\data\\amazon\\"
DATASET = r"meta_Software.json"

def checkAttr():
    '''
        check whether ASIN of the item is existing.
        
        Result: 
            all of the items exist ASIN of the properties.
    '''
    with open(DIR_PATH + DATASET, "r") as f:
        data = pd.read_json(f, orient="records", typ="series", lines=True)
    
        for item in data:
            if not all(key in item for key in ["asin", "also_view", "also_buy", "category", "price"]):
                print(item)

def reduceAttr(output_file):
    '''
        保留需要的商品屬性, asin, also_view, also_buy, category, price.
        also_view和also_buy有多個以上時, 用空白隔開
    '''
    
    with open(DIR_PATH + DATASET, "r") as f:
        data = pd.read_json(f, orient="records", typ="series", lines=True, dtype={"category":list})

        output = open(DIR_PATH + output_file, "w")
        for item in data:
            also_view = " ".join(item["also_view"])
            also_buy = " ".join(item["also_buy"])
            category = " ".join([preprocessingText(string) for string in item["category"]])
            price = item["price"][1:] if item["price"] != "" and item["price"][0] == "$" else ""

            if also_view and also_buy and category and price:
                row = "{asin},{also_view},{also_buy},{category},{price}".format(
                    asin = item["asin"], 
                    also_view = also_view, 
                    also_buy = also_buy, 
                    category = category, 
                    price = price.replace(",", ""))
                output.write(row + "\n")
        output.close()
checkAttr()
reduceAttr("extraction.csv")

### 需要保留的商品屬性
- asin
- also_view
- also_buy
- category
- price

In [ ]:
with open(DIR_PATH + DATASET, "r") as f:
    data = pd.read_json(f, orient="records", typ="series", lines=True)
    count = 0
    for item in data:
        if item["asin"] == "0078605407":
            count += 1
            also_view = " ".join(item["also_view"])
            also_buy = " ".join(item["also_buy"])
            category = " ".join([preprocessingText(string) for string in item["category"]])
            price = item["price"][1:] if item["price"] != "" and item["price"][0] == "$" else ""
            
            print("{asin},{also_view},{also_buy},{category},{price}".format(
                    asin = item["asin"], 
                    also_view = also_view, 
                    also_buy = also_buy, 
                    category = category, 
                    price = str(price)))
#print("{0}/{1}".format(count, data.size))

In [ ]:
with open(DIR_PATH + "extraction.csv", "r") as f:
    count = 0
    for line in f:
        count = count + 1
    print(count)

In [10]:
def read_items(filename):
    dataset = dict()
    with open(filename) as file:
        next(file)
        for line in file:
            asin, also_view, also_buy, price = line.split(",")
            dataset[asin] = dict()
            dataset[asin]["also_view"] = also_view.split(" ")
            dataset[asin]["also_buy"] = also_buy.split(" ")
            dataset[asin]["price"] = float(price)

    return dataset

def sample_items(dataset:dict, root, num_substituion=0, num_complements=0, picked = []):
    
    def _bfs_sample(dataset, root, num, relation:str, picked = picked):
        
        if relation not in ['also_buy', 'also_view']:
            raise ValueError("Parameter of relation is an error")

        queue = []
        if root not in picked:
            picked.append(root)
        queue.append(root)

        while len(queue) != 0 and num > 0:
            for i in dataset[root][relation]:
                if i in dataset and i not in picked:
                    queue.append(i)
            
            # pop first element
            root = queue.pop(0)

            if root not in picked:
                picked.append(root)
                num -= 1
        return num

    num_substituion = _bfs_sample(dataset, root, num_substituion, "also_view", picked)
    num_complements = _bfs_sample(dataset, root, num_complements, "also_buy", picked)
    return num_substituion, num_complements
    
    
# print("asin: " + max_freq_asin)
# print("also_view: {0}".format(dataset[max_freq_asin]["also_view"]))
# print("also_buy: {0}".format(dataset[max_freq_asin]["also_buy"]))
# print("price: {0}".format(dataset[max_freq_asin]["price"]))
# print("freq: {0}".format(dataset[max_freq_asin]["freq"]))

In [11]:
DIR_PATH = r".\\data\\amazon\\"
FILE = r"preprocessed_Software.csv"

dataset = read_items(DIR_PATH + FILE)


In [12]:
result = []
num_substituion, num_complements = sample_items(dataset=dataset, root="B000VYIUJS", num_substituion=9, num_complements=10, picked=result)

# for asin in dataset.keys():
#     if len(result) <= 100:
#         num_substituion, num_complements = sample_items(dataset=dataset, root=asin, num_substituion=num_substituion, num_complements=num_complements, picked=result)

# for i in result:
#     attr = dataset[i]
#     also_view = set(attr["also_view"])
#     also_buy = set(attr["also_buy"])
#     inter = also_buy.intersection(also_view)
#     print("{}: (also_view, {}), (also_buy, {}), (inter, {})".format(i, len(also_buy), len(also_view), len(inter)))

with open(DIR_PATH + "/sample_items.csv", "w") as file:
    file.write("asin,also_view,also_buy,price\n")
    for asin in result:
        also_view = [a for a in dataset[asin]["also_view"] if a in result]
        also_buy = [a for a in dataset[asin]["also_buy"] if a in result]
        
        file.write("{},{},{},{}\n".format(
            asin,
            " ".join(also_view),
            " ".join(also_buy),
            dataset[asin]["price"]
        ))

### Randomly Generate topic of the items

In [13]:
from random import random
def generate(numberTopics):
    topic = [random() for t in range(numberTopics)]
    norm = sum(topic)
    normTopic = [t/norm for t in topic]
    
    return normTopic

NUM_TOPIC = 5
DIR_PATH = r"C:\\Users\\5F19 DS Lab\\Desktop\\HeJi\\論文實驗\\data\\amazon"
smaple_file = DIR_PATH + "/sample_items.csv"
topic_file = DIR_PATH + "/items_with_" + str(NUM_TOPIC) + "_topic.csv"

with open(smaple_file, "r") as file:
    output = open(topic_file, "w")
    line = file.readline()
    for line in file.readlines():
        asin, *_ = line.split(",")
        output_line = asin + "," + " ".join([str(t) for t in generate(NUM_TOPIC)])
        output.write(output_line + "\n")
    output.close()

In [ ]:
generate(NUM_TOPIC)

[0.13496512853076195,
 0.0038977556344887746,
 0.2643477716479573,
 0.2544791427993728,
 0.34231020138741924]